<h4> Importing library

In [10]:
import pandas as pd 
import numpy as np 
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import requests
import folium
import datetime

In [11]:
address = "piscataway,NJ"
geolocation = Nominatim(user_agent="foursquare_agent")
location = geolocation.geocode(address)

<h4> Masking the client id and secret as this is sensitive information
<h4> Please note that this file will not run unless the client id and secret is provided

In [92]:
client_id = #Not provided publicaly 
client_secret = #Not provided publicaly 
limit = 100 
radius = 1000 

<h4> Figure out all stores in the neighbuorhood which sell Acrylic paints

In [93]:
query="Acrylic paint"
v=datetime.datetime.today().strftime('%Y%m%d')
ll = f"{location.latitude},{location.longitude}"
venue_search = "https://api.foursquare.com/v2/venues/search"
venue_search_endpoint=f"{venue_search}?client_id={client_id}&client_secret={client_secret}&v={v}&ll={ll}&query={query}"

In [139]:
#figure out all the stores nearby which sell acrlyic paints 
#figure out who liked the venue 
#how many of the users were professional painters 

res = requests.get(url=venue_search_endpoint).json()
res["response"]["venues"]

[{'id': '4fd8aff3e4b04e404e236432',
  'name': 'Sherwin-Williams Paint Store',
  'location': {'address': '1623 Stelton Rd',
   'lat': 40.535708621893335,
   'lng': -74.42030492962907,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.535708621893335,
     'lng': -74.42030492962907}],
   'distance': 4043,
   'postalCode': '08854',
   'cc': 'US',
   'city': 'Piscataway',
   'state': 'NJ',
   'country': 'United States',
   'formattedAddress': ['1623 Stelton Rd',
    'Piscataway, NJ 08854',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d112951735',
    'name': 'Hardware Store',
    'pluralName': 'Hardware Stores',
    'shortName': 'Hardware',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/hardware_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1586736006',
  'hasPerk': False},
 {'id': '587408558cb34e6706d33b60',
  'name': 'Yospin Paint',
  'location': {'address': '386 North Ave',
   'lat': 40.590849,
   'lng': -74.465003,
 

<h4> Clean up data and create a search list with relevant information

In [140]:
search_list=[]
for items in res["response"]["venues"]:
    search_data={}
    search_data["id"]=items["id"]
    search_data["name"]=items["name"]
    search_data["lat"]=items["location"]["lat"]
    search_data["lng"]=items["location"]["lng"]
    search_data["distance"]=items["location"]['distance']
    search_data["postalcode"]=items["location"]['formattedAddress'][1]
    search_data["categories"]=items["categories"]
    search_list.append(search_data)

In [141]:
df1  = pd.DataFrame(search_list)
list1=[]
for row,cols in df1.iterrows():
    try:
        list1.append(cols[6][0]["name"])
    except:
        list1.append("NA")
df1["category_name"]=list1

In [142]:
df2 = df1.drop("categories",axis=1)

In [143]:
df2.head()

,id,name,lat,lng,distance,postalcode,category_name
0,4fd8aff3e4b04e404e236432,Sherwin-Williams Paint Store,40.535709,-74.420305,4043,"Piscataway, NJ 08854",Hardware Store
1,587408558cb34e6706d33b60,Yospin Paint,40.590849,-74.465003,4964,"Dunellen, NJ 08812",Home Service
2,5259304a11d2860074ed2d8c,Bound Brook Paint,40.561375,-74.533928,5983,"Bound Brook, NJ 08805",Hardware Store
3,51699e3ce4b020b8b208f1fe,Ricciardi Brothers Paint,40.542942,-74.403397,5311,United States,Hardware Store
4,50a119e4498efdffc83a8085,Paint Tek,40.595973,-74.470311,5546,"Dunellen, NJ 08812",Miscellaneous Shop


<h4> Make a user call to determine number of likes 
<h4> For this project we are interested in understanding the number of likes 

In [144]:
#figure out the users who liked these areas 
user_list = []

v=datetime.datetime.today().strftime('%Y%m%d')
for items in df2["id"]:
    user_dict={}
    user_url = f"https://api.foursquare.com/v2/venues/{items}/likes?client_id={client_id}&client_secret={client_secret}&v={v}"
    res = requests.get(url=user_url).json()
    user_dict["id"]=items
    user_dict["likes"]=res["response"]["likes"]["count"]
#     user_like=[]
#     try:
#         for users in res["response"]["likes"]["items"]:
#             user_like.append(users["id"])   
#     except:
#         user_like.append("NA")
#     user_dict["user_id"]=user_like
    user_list.append(user_dict)
# user_list

In [200]:
df3 = pd.DataFrame(user_list)
df3.head()

,id,likes
0,4fd8aff3e4b04e404e236432,0
1,587408558cb34e6706d33b60,1
2,5259304a11d2860074ed2d8c,0
3,51699e3ce4b020b8b208f1fe,0
4,50a119e4498efdffc83a8085,0


<h4> Merge the two dataframes to create final dataframe

In [201]:
final_tally = pd.merge(df2,df3,on="id",how="inner")
final_tally

,id,name,lat,lng,distance,postalcode,category_name,likes
0,4fd8aff3e4b04e404e236432,Sherwin-Williams Paint Store,40.535709,-74.420305,4043,"Piscataway, NJ 08854",Hardware Store,0
1,587408558cb34e6706d33b60,Yospin Paint,40.590849,-74.465003,4964,"Dunellen, NJ 08812",Home Service,1
2,5259304a11d2860074ed2d8c,Bound Brook Paint,40.561375,-74.533928,5983,"Bound Brook, NJ 08805",Hardware Store,0
3,51699e3ce4b020b8b208f1fe,Ricciardi Brothers Paint,40.542942,-74.403397,5311,United States,Hardware Store,0
4,50a119e4498efdffc83a8085,Paint Tek,40.595973,-74.470311,5546,"Dunellen, NJ 08812",Miscellaneous Shop,0
5,507c811300ca3bfe2a84482d,Sherwin-Williams Commercial Paint Store,40.596568,-74.481430,5749,"Green Brook Township, NJ 08812",Hardware Store,0
6,5b1287cd4f0e58002c089a40,New Jersey Paint Pros,40.536193,-74.396442,5993,"Edison, NJ 08817",Home Service,0
7,51efdd0e50195cbae90ea3fd,Warrenville Paint and Hardware,40.618378,-74.494794,8388,"Warren, NJ 07059",Hardware Store,0
8,4fc77cefe4b03776d9ebec3d,Sherwin-Williams Commercial Paint Store,40.607404,-74.469366,6812,"Green Brook Township, NJ 08812",Hardware Store,0
9,504f442ae4b022569424b407,Sherwin-Williams Paint Store,40.594952,-74.399701,7800,"South Plainfield, NJ 07080",Hardware Store,1


In [202]:
def num_of_like(value):
    if value == 0:
        return(2)
    if value == 1:
        return(7)
    if value >=2:
        return(14)

<h4> Map the neighbourhood , The size of the circle tells us the number of likes . Bigger the circle , better is the feedback

In [203]:
paint_map = folium.Map(location=[final_tally.loc[0,"lat"],final_tally.loc[0,"lng"]], zoom_start=13)
for lat, lng, catname,likes,placename in zip(final_tally["lat"], final_tally["lng"], final_tally["category_name"],final_tally["likes"],final_tally["name"]):
    label = str(catname) + " :likes " + str(likes)
    folium.features.CircleMarker(
        [lat, lng],
        radius=num_of_like(likes) ,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(paint_map)
paint_map

<h4>One hot encoding and get complete clustering

In [227]:
#clustering based on the category_type
paint_onehot= pd.get_dummies(final_tally[["category_name"]])

In [228]:
paint_onehot.insert(loc=0,column="name",value=final_tally["name"])
paint_onehot.insert(loc=1,column="lat",value=final_tally["lat"])
paint_onehot.insert(loc=1,column="lng",value=final_tally["lng"])
paint_onehot.insert(loc=1,column="likes",value=final_tally["likes"])
paint_onehot

,name,likes,lng,lat,category_name_Arts & Crafts Store,category_name_Building,category_name_Construction & Landscaping,category_name_Design Studio,category_name_Hardware Store,category_name_Home Service,category_name_Miscellaneous Shop,category_name_NA,category_name_School
0,Sherwin-Williams Paint Store,0,-74.420305,40.535709,0,0,0,0,1,0,0,0,0
1,Yospin Paint,1,-74.465003,40.590849,0,0,0,0,0,1,0,0,0
2,Bound Brook Paint,0,-74.533928,40.561375,0,0,0,0,1,0,0,0,0
3,Ricciardi Brothers Paint,0,-74.403397,40.542942,0,0,0,0,1,0,0,0,0
4,Paint Tek,0,-74.470311,40.595973,0,0,0,0,0,0,1,0,0
5,Sherwin-Williams Commercial Paint Store,0,-74.481430,40.596568,0,0,0,0,1,0,0,0,0
6,New Jersey Paint Pros,0,-74.396442,40.536193,0,0,0,0,0,1,0,0,0
7,Warrenville Paint and Hardware,0,-74.494794,40.618378,0,0,0,0,1,0,0,0,0
8,Sherwin-Williams Commercial Paint Store,0,-74.469366,40.607404,0,0,0,0,1,0,0,0,0
9,Sherwin-Williams Paint Store,1,-74.399701,40.594952,0,0,0,0,1,0,0,0,0


In [229]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=4,random_state=0)
X=paint_onehot.drop(["name","lat","lng","likes"],axis=1)
km.fit(X)
labels=km.labels_
center = km.cluster_centers_
km.labels_[0:75] 

array([1, 2, 1, 1, 0, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 0, 0, 0, 1, 0, 3, 1,
       1, 1, 1, 0, 1, 1, 1, 1])

<h4> Merge cluster labels with the original dataframe

In [230]:
paint_onehot.insert(loc=1,value=km.labels_,column="cluster_label")
paint_onehot["lat"]=paint_onehot["lat"].astype(float)
paint_onehot["lng"]=paint_onehot["lng"].astype(float)
paint_onehot.head()

,name,cluster_label,likes,lng,lat,category_name_Arts & Crafts Store,category_name_Building,category_name_Construction & Landscaping,category_name_Design Studio,category_name_Hardware Store,category_name_Home Service,category_name_Miscellaneous Shop,category_name_NA,category_name_School
0,Sherwin-Williams Paint Store,1,0,-74.420305,40.535709,0,0,0,0,1,0,0,0,0
1,Yospin Paint,2,1,-74.465003,40.590849,0,0,0,0,0,1,0,0,0
2,Bound Brook Paint,1,0,-74.533928,40.561375,0,0,0,0,1,0,0,0,0
3,Ricciardi Brothers Paint,1,0,-74.403397,40.542942,0,0,0,0,1,0,0,0,0
4,Paint Tek,0,0,-74.470311,40.595973,0,0,0,0,0,0,1,0,0


In [231]:
def return_color(value):
    if value==0:
        return("red")
    elif value==1:
        return("blue")
    elif value==2:
        return("yellow")
    elif value==3:
        return("green")
    else:
        return("black")

In [232]:
paint_map_cluster = folium.Map(location=[paint_onehot.loc[0,"lat"],paint_onehot.loc[0,"lng"]], zoom_start=13)
for lat, lng, label1,likes in zip(paint_onehot["lat"], paint_onehot["lng"],paint_onehot["cluster_label"],paint_onehot["likes"]):
    label = "cluster: " + str(label1)
    folium.features.CircleMarker(
        [lat, lng],
        radius=num_of_like(likes) ,
        color=return_color(label1),
        popup=label,
        fill = True,
        fill_color=return_color(label1),
        fill_opacity=0.6
    ).add_to(paint_map_cluster)
paint_map_cluster

<h4> End of workbook 